<a href="https://colab.research.google.com/github/shishir-py/Classification-using-Deep-learning/blob/main/intent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
train_path="/content/drive/MyDrive/Colab Notebooks/Project/Review/intent analysis/Bitext_Sample_Customer_Service_Training_Dataset.csv"
valid_path="/content/drive/MyDrive/Colab Notebooks/Project/Review/intent analysis/Bitext_Sample_Customer_Service_Validation_Dataset.csv"
test_data="/content/drive/MyDrive/Colab Notebooks/Project/Review/intent analysis/Bitext_Sample_Customer_Service_Testing_Dataset.csv"
df_train=pd.read_csv(train_path)
df_test=pd.read_csv(test_data)
df_val=pd.read_csv(valid_path)


In [10]:
df_train.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order I made?,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,I need assistance canceling the last order I h...,cancel_order,ORDER,B
3,problem with canceling the order I made,cancel_order,ORDER,B
4,I don't know how to cancel the order I made,cancel_order,ORDER,B


In [12]:
df_train.columns

Index(['utterance', 'intent', 'category', 'tags'], dtype='object')

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


# Split data into X and y
X_train = df_train["utterance"].values
y_train = df_train["intent"].values
X_test = df_test["utterance"].values
y_test = df_test["intent"].values
X_val = df_val["utterance"].values
y_val = df_val["intent"].values

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)

# Pad sequences
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_val = pad_sequences(X_val, maxlen=max_len)

# Encode categorical labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

# Define model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_len))
model.add(Flatten())
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(len(label_encoder.classes_), activation="softmax"))

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64)

# Evaluate model on test set
loss, acc = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", acc)


Epoch 1/50
103/103 [==============================] - 1s 6ms/step - loss: 3.2131 - accuracy: 0.0650 - val_loss: 2.9894 - val_accuracy: 0.1284
Epoch 2/50
103/103 [==============================] - 1s 7ms/step - loss: 2.4791 - accuracy: 0.2811 - val_loss: 1.9000 - val_accuracy: 0.4731
Epoch 3/50
103/103 [==============================] - 1s 8ms/step - loss: 1.3223 - accuracy: 0.6899 - val_loss: 0.9881 - val_accuracy: 0.7934
Epoch 4/50
103/103 [==============================] - 1s 8ms/step - loss: 0.7156 - accuracy: 0.8500 - val_loss: 0.6151 - val_accuracy: 0.8716
Epoch 5/50
103/103 [==============================] - 1s 8ms/step - loss: 0.4550 - accuracy: 0.9020 - val_loss: 0.4472 - val_accuracy: 0.8985
Epoch 6/50
103/103 [==============================] - 1s 9ms/step - loss: 0.3182 - accuracy: 0.9307 - val_loss: 0.3450 - val_accuracy: 0.9181
Epoch 7/50
103/103 [==============================] - 1s 9ms/step - loss: 0.2323 - accuracy: 0.9524 - val_loss: 0.2733 - val_accuracy: 0.9499
Epoch 

In [29]:
# Tokenize and pad test data
X_test = tokenizer.texts_to_sequences(df_test["utterance"].values)
X_test = pad_sequences(X_test, maxlen=max_len)

# Predict on test data
y_pred = model.predict(X_test)

# Decode categorical labels
y_pred = np.argmax(y_pred, axis=1)
y_pred = label_encoder.inverse_transform(y_pred)

# Print predicted labels
print(y_pred)


26/26 [==============================] - 0s 2ms/step
['cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'change_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'switch_account' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order'

In [30]:
len(y_pred)

818

In [31]:
df_test.shape

(818, 4)

In [39]:
results_df = pd.DataFrame({"Actual Intent": df_test['intent'], "Predicted Intent": y_pred})
print(results_df)


    Actual Intent Predicted Intent
0    cancel_order     cancel_order
1    cancel_order     cancel_order
2    cancel_order     cancel_order
3    cancel_order     cancel_order
4    cancel_order     cancel_order
..            ...              ...
813  track_refund     track_refund
814  track_refund     track_refund
815  track_refund     track_refund
816  track_refund     track_refund
817  track_refund     track_refund

[818 rows x 2 columns]


In [41]:
values_match = results_df['Actual Intent'] == results_df['Predicted Intent']

# print the results
print(values_match)

0      True
1      True
2      True
3      True
4      True
       ... 
813    True
814    True
815    True
816    True
817    True
Length: 818, dtype: bool


In [42]:
true_count = 0
false_count = 0

for i in range(len(results_df)):
    if results_df["Actual Intent"][i] == results_df["Predicted Intent"][i]:
        true_count += 1
    else:
        false_count += 1

print("Number of correct predictions:", true_count)
print("Number of incorrect predictions:", false_count)


Number of correct predictions: 806
Number of incorrect predictions: 12


In [44]:
# Save model to file
model.save('intent_classifier.h5')


In [45]:
import os

print(os.getcwd())  # print current working directory

# list contents of current working directory
for file in os.listdir():
    print(file)


/content
.config
intent_classifier.h5
drive
sample_data


In [48]:
import pickle

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [51]:
# Import necessary libraries
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import pandas as pd
import pickle

# Load saved model
model = load_model("/content/intent_classifier.h5")

# Load tokenizer
with open("/content/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load label encoder
with open("/content/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Define function to make predictions
def predict_intent(utterance):
    # Tokenize text and pad sequence
    seq = tokenizer.texts_to_sequences([utterance])
    padded_seq = pad_sequences(seq, maxlen=100)

    # Make prediction
    pred = model.predict(padded_seq)[0]

    # Get predicted label and confidence score
    label = label_encoder.inverse_transform([np.argmax(pred)])[0]
    confidence = round(np.max(pred), 4)

    # Return prediction results
    return label, confidence

# Test function with sample utterance
utterance = "What's the trending product today?"
utterance1 = "how can i cancle product?"

label, confidence = predict_intent(utterance)
label1, confidence1 = predict_intent(utterance1)

print(f"Predicted intent: {label}, Confidence: {confidence}")
print(f"Predicted intent: {label1}, Confidence: {confidence1}")



1/1 [==============================] - 0s 22ms/step
Predicted intent: place_order, Confidence: 0.7401999831199646
Predicted intent: change_order, Confidence: 0.8996000289916992
